In [1]:
import pandas as pd
import numpy as np

In [4]:
from dotenv import load_dotenv
import os

# Load file .env
load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [2]:
books = pd.read_csv('D://Workspace//LLM//Book Recommendation//dataset//books_with_categories.csv')
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction


### **1. Extracting emotions from book descriptions**

In [6]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = 'cpu')
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528673090040684},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419779404997826},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.0016119909705594182},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

In [7]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [8]:
classifier(books['description'][0])

[[{'label': 'fear', 'score': 0.6548418998718262},
  {'label': 'neutral', 'score': 0.16985182464122772},
  {'label': 'sadness', 'score': 0.11640854179859161},
  {'label': 'surprise', 'score': 0.020700592547655106},
  {'label': 'disgust', 'score': 0.01910073682665825},
  {'label': 'joy', 'score': 0.015161239542067051},
  {'label': 'anger', 'score': 0.00393515033647418}]]

It can be seen that the first sentence containing 'astonishingly' leans towards astonishment, while other sentences may carry emotions of fear, joy, or neutrality which can lead to inaccurate or unrepresentative results. To address this, we propose breaking down the descriptions into individual sentences based on periods, using an emotion classification model to predict labels such as anger, disgust, fear, joy, neutral, sadness, and surprise for each sentence, and then selecting the highest scoring value for each type of emotion, ensuring that the most prominent emotion is accurately reflected and fairly evaluating each sentence independently before compiling the results for each book based on its ISBN.

In [9]:
books['description'][0].split('.')[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [10]:
classifier(books['description'][0].split('.')[0])

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.14038576185703278},
  {'label': 'fear', 'score': 0.06816209107637405},
  {'label': 'joy', 'score': 0.04794240742921829},
  {'label': 'anger', 'score': 0.00915635284036398},
  {'label': 'disgust', 'score': 0.0026284719351679087},
  {'label': 'sadness', 'score': 0.0021221607457846403}]]

In [11]:
sequence = books['description'][0].split('.')
res = classifier(sequence)

In [12]:
print(sequence[4])
res[4]

 He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption


[{'label': 'sadness', 'score': 0.9671575427055359},
 {'label': 'neutral', 'score': 0.015104170888662338},
 {'label': 'disgust', 'score': 0.006480592768639326},
 {'label': 'fear', 'score': 0.005393994972109795},
 {'label': 'surprise', 'score': 0.0022869433742016554},
 {'label': 'anger', 'score': 0.0018428886542096734},
 {'label': 'joy', 'score': 0.001733880490064621}]

In [13]:
sorted(res[0], key=lambda x: x['label'])

[{'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.0026284719351679087},
 {'label': 'fear', 'score': 0.06816209107637405},
 {'label': 'joy', 'score': 0.04794240742921829},
 {'label': 'neutral', 'score': 0.14038576185703278},
 {'label': 'sadness', 'score': 0.0021221607457846403},
 {'label': 'surprise', 'score': 0.7296027541160583}]

### **2. Test for 5 or 10 descriptions of the dataset**

In [14]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# Function to calculate max emotion in each sentence in a description
def get_max_emotion_scores(predictions):
  emotion_scores = {label: [] for label in emotion_labels}
  for pred in predictions:
    sorted_predictions = sorted(pred, key=lambda x: x['label'])
    for idx, label in enumerate(emotion_labels):
      emotion_scores[label].append(sorted_predictions[idx]['score'])
    return {label: np.max(score) for label, score in emotion_scores.items()}

In [15]:
isbn_tmp = []
emotion_scores = {label: [] for label in emotion_labels}

for i in range(5):
  isbn_tmp.append(books['isbn13'][i])
  sentences_in_each_descriptions = books['description'][i].split('.')
  predictions = classifier(sentences_in_each_descriptions)
  max_emotion_scores = get_max_emotion_scores(predictions)
  for label in emotion_labels:
    emotion_scores[label].append(max_emotion_scores[label])

In [16]:
emotion_scores

{'anger': [np.float64(0.009156372398138046),
  np.float64(0.005966845899820328),
  np.float64(0.04130077362060547),
  np.float64(0.016036225482821465),
  np.float64(0.013624386861920357)],
 'disgust': [np.float64(0.002628477057442069),
  np.float64(0.0028870627284049988),
  np.float64(0.024568283930420876),
  np.float64(0.06069513037800789),
  np.float64(0.12224264442920685)],
 'fear': [np.float64(0.06816234439611435),
  np.float64(0.003809872781857848),
  np.float64(0.10406067222356796),
  np.float64(0.0016916367458179593),
  np.float64(0.09504339098930359)],
 'joy': [np.float64(0.047942597419023514),
  np.float64(0.7044220566749573),
  np.float64(0.7672385573387146),
  np.float64(0.16175726056098938),
  np.float64(0.008335679769515991)],
 'neutral': [np.float64(0.14038586616516113),
  np.float64(0.21775951981544495),
  np.float64(0.04217583313584328),
  np.float64(0.7326849102973938),
  np.float64(0.2726137042045593)],
 'sadness': [np.float64(0.002122163074091077),
  np.float64(0.004

### **3. Test for the dataset**

In [17]:
from tqdm import tqdm

emotional_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
emotion_scores_list = {label: [] for label in emotional_labels}
isbn_list = []

for i in tqdm(range(len(books))):
  isbn_list.append(books['isbn13'][i])
  sentences = books['description'][i].split('.')
  predictions = classifier(sentences)
  max_emotion_scores = get_max_emotion_scores(predictions)
  for label in emotional_labels:
    emotion_scores_list[label].append(max_emotion_scores[label])

100%|██████████| 5197/5197 [39:51<00:00,  2.17it/s]


In [18]:
df_emotions = pd.DataFrame(emotion_scores_list)
df_emotions['isbn13'] = isbn_list
df_emotions.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.009156,0.002628,0.068162,0.047943,0.140386,0.002122,0.729602,9780002005883
1,0.005967,0.002887,0.003810,0.704422,0.217760,0.004509,0.060646,9780002261982
2,0.041301,0.024568,0.104061,0.767239,0.042176,0.010860,0.009796,9780006178736
3,0.016036,0.060695,0.001692,0.161757,0.732685,0.020988,0.006147,9780006280897
4,0.013624,0.122243,0.095043,0.008336,0.272614,0.475880,0.012260,9780006280934


In [19]:
books = pd.merge(books, df_emotions, on='isbn13')

In [20]:
books.sample(5)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
5040,9781857231359,185723135X,Use of Weapons,Iain Banks,Science fiction,NaN,The man known as Cheradenine Zakalwe was one o...,1992.0,4.19,411.0,...,Use of Weapons,9781857231359 The man known as Cheradenine Zak...,Nonfiction,0.044430,0.870222,0.011635,0.004855,0.056623,0.007436,0.004800
1602,9780373274611,0373274610,The Bluest Eyes in Texas,Marilyn Pappano,Fiction,http://books.google.com/books/content?id=K9nct...,Army hero Logan Marshall would avenge his fost...,2005.0,3.67,249.0,...,The Bluest Eyes in Texas,9780373274611 Army hero Logan Marshall would a...,Fiction,0.932244,0.030714,0.001491,0.002501,0.022231,0.009989,0.000830
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,...,The Family Way,9780007151240 It should be the most natural th...,Fiction,0.079929,0.037985,0.016266,0.132576,0.525620,0.018228,0.189396
40,9780007120819,0007120818,The Big Four,Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=wcOQU...,A ruthless international cartel seeks world do...,2002.0,3.59,272.0,...,The Big Four,9780007120819 A ruthless international cartel ...,Nonfiction,0.563667,0.157762,0.039791,0.008598,0.204828,0.014475,0.010879
1391,9780330351690,0330351699,Into the Wild,Jon Krakauer,True Crime,http://books.google.com/books/content?id=X7M1D...,"Using the true story of a young man, who in 19...",1998.0,3.97,206.0,...,Into the Wild,9780330351690 Using the true story of a young ...,Nonfiction,0.011372,0.437456,0.039939,0.003498,0.184157,0.312138,0.011439


In [21]:
books.to_csv("D://Workspace//LLM//Book Recommendation//dataset//books_with_emotions.csv", index=False)